# Part 3 - Exercises - Interactivity in Vega-Altair
In this section, you will get hands-on experience adding interactivity to static Vega-Altair charts.  You will practice adding selections to charts, styling marks with conditional encodings, and using the `transform_filter` method with selections.


## Imports

In [ ]:
import altair as alt
import pandas as pd
from vega_datasets import data
print("The installed Vega-Altair version is " + alt.__version__)

## Cars exercise

Let's look again at the final example from the multi-view selection section of Part 3. That example creates a scatter plot of horsepower and miles per gallon along with an interval selection, where points can be selected by dragging a rectangle. The selection is used to filter that dataset in the bar chart below.

Remove the interval selection and try to reverse the interaction pattern so that clicking on a bar of the bar chart highlights the bar and the corresponding points in the scatter plot.

Here is a static snapshot of the desired chart after clicking on the "Japan" bar in the bar chart.

![](../resources/images/part3/cars1.png)
<details>
  <summary>(Show Answer)</summary>

  ```python
cars = data.cars()

selection = alt.selection_point(fields=["Origin"])

scatter = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(selection, 'Origin', alt.value('lightgray'))
)

bars = alt.Chart(cars).mark_bar().encode(
    alt.X('count(Origin)').scale(domain=[0,260]),
    alt.Y('Origin').scale(domain=["Europe", "Japan", "USA"]),
    alt.Color('Origin'),
).add_params(
    selection
)

scatter & bars
  ```
</details>

---
Now, switch to filtering the points in the scatter plot rather than highlighting them.


Here is a static snapshot of the desired chart after clicking on the "USA" bar in the bar chart.


![](../resources/images/part3/cars2.png)
<details>
  <summary>(Show Answer)</summary>

  ```python
cars = data.cars()

selection = alt.selection_point(fields=["Origin"])

scatter = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    alt.Color('Origin')
).transform_filter(
    selection
)

bars = alt.Chart(cars).mark_bar().encode(
    alt.X('count(Origin)').scale(domain=[0,260]),
    alt.Y('Origin').scale(domain=["Europe", "Japan", "USA"]),
    color=alt.condition(selection, 'Origin', alt.value('lightgray'))
).add_params(
    selection
)

scatter & bars
  ```
</details>

## Airports exercise
Next, we'll look at an example that is loosely based on a more involved [Airports example](https://altair-viz.github.io/gallery/airport_connections.html) from the gallery.  

This dataset will use the `airports` dataset from `vega_datasets`, which includes the location of airports in the United States.

We begin with the following static chart of the positions of airports

In [ ]:
airports = data.airports()
airports.head()

In [ ]:
states = alt.topo_feature(data.us_10m.url, feature="states")

background = alt.Chart(states).mark_geoshape(
    fill="lightgray",
    stroke="white"
).properties(
    width=750,
    height=500
).project("albersUsa")

points = alt.Chart(airports).mark_circle().encode(
    alt.Latitude("latitude:Q"),
    alt.Longitude("longitude:Q"),
)

background+points

Add an interactive component to the chart as follows:

* Based on the user's mouse, select the nearest airport as well as all other airports within the same state.  Display those selected airports with an opacity value of `1`, and all non-selected airports with an opacity value of `0`.

<details>
  <summary>(Show Image)</summary>
  Here is a static snapshot of the desired chart when the mouse cursor hovers over Kansas:  
  <img src="../resources/images/part3/airports_state.png">
</details>

<details>
  <summary>(Show Answer)</summary>

  ```python
airports = data.airports()
states = alt.topo_feature(data.us_10m.url, feature="states")

highlight = alt.selection_point(on="pointerover", nearest=True, fields=["state"])

background = alt.Chart(states).mark_geoshape(
    fill="lightgray",
    stroke="white"
).properties(
    width=750,
    height=500
).project("albersUsa")

points = alt.Chart(airports).mark_circle().encode(
    alt.Latitude("latitude:Q"),
    alt.Longitude("longitude:Q"),
    opacity=alt.condition(highlight, alt.value(1), alt.value(0))
).add_params(
    highlight
)

background+points
  ```
</details>


### Challenge exercise:
* Instead of selecting by state, select by the nearest 5 degrees of latitude.  (Use a `transform_calculate` to create a new column that is rounded to the nearest 5 degrees.)
* Also include a nominal color encoding based on these degrees of latitude.
* What is the eastern-most US airport in the same latitude band as northern Washington state?  (Use a tooltip.)

<details>
  <summary>(Show Image)</summary>
  Here is a static snapshot of the described chart. 
  <img src="../resources/images/part3/airports_lat.png">
</details>

<details>
  <summary>(Show Answer)</summary>

  ```python
airports = data.airports()
states = alt.topo_feature(data.us_10m.url, feature="states")

highlight = alt.selection_point(on="pointerover", nearest=True, fields=["lat5"])

background = alt.Chart(states).mark_geoshape(
    fill="lightgray",
    stroke="white"
).properties(
    width=750,
    height=500
).project("albersUsa")

points = alt.Chart(airports).transform_calculate(
    lat5 = "round(datum.latitude/5)*5"
).mark_circle().encode(
    alt.Latitude("latitude:Q"),
    alt.Longitude("longitude:Q"),
    alt.Color("lat5:N"),
    opacity=alt.condition(highlight, alt.value(1), alt.value(0)),
    tooltip=["iata", "name"]
).add_params(
    highlight
)

background+points
  ```
</details>


If you finish early, check out the more complicated [Airport Connections](https://altair-viz.github.io/gallery/airport_connections.html) example from the gallery and see how much of it you can recognize and what parts are unfamiliar.

## Stocks exercise

Our goal in this exercise is to plot stock prices for various companies using a line chart.  The *interactive* component of this chart will provide the ability to emphasize individual companies within the chart.  In building up to the working example, we will cover several *gotchas* along the way.  

**Warning: Contains spoilers**.  This exercise is based on the [Multi-Line Highlight](https://altair-viz.github.io/gallery/multiline_highlight.html) example from the gallery, but do not click the link unless you want a spoiler!

* Load the `stocks` dataset from vega_datasets.

In [ ]:
source = data.stocks()
source


* Make a line chart using the "date" field for the x-axis encoding and using the "price" field for the y-axis encoding.
* What is strange about the appearance of this chart?
* Can you recognize why the chart looks the way it does, and what could be done to make it look more natural?  (It will probably be necessary to take a look at the stocks data to answer this question.)

<details>
  <summary>(Show Image)</summary>
  <img src="../resources/images/part3/stocks1.png">
</details>

<details>
  <summary>(Show Answer)</summary>

  ```python
alt.Chart(source).mark_line().encode(
    alt.X("date"),
    alt.Y("price"),
)
  ```
</details>

* Add a color encoding corresponding to the stock symbol.  Do you see why this completely changes the shapes of the plotted lines?

<details>
  <summary>(Show Image)</summary>
  <img src="../resources/images/part3/stocks2.png">
</details>

<details>
  <summary>(Show Answer)</summary>

  ```python
alt.Chart(source).mark_line().encode(
    alt.X("date"),
    alt.Y("price"),
    alt.Color("symbol")
)
  ```
</details>

* Create a point selection parameter `highlight`, add the parameter to this chart, and use the parameter to filter the data, using `transform_filter`.
* The variable name `highlight` will need to appear three total times (including the definition), what are those three?
* Try clicking a point on the chart.
* Comment 1:  You need to click on a point corresponding to a data point (as opposed to a line connecting two data points).
* Comment 2:  You can get back to the original view by clicking on an empty portion of the chart.
* If things are set up "correctly", all the lines should disappear, and we should get a zoomed in view on what looks like an empty region.
* Rhetorical question: What should a line chart display when there is only one point?

<details>
  <summary>(Show Image)</summary>
  <img src="../resources/images/part3/stocks3.png">
</details>

<details>
  <summary>(Show Answer)</summary>

  ```python
highlight = alt.selection_point()

alt.Chart(source).transform_filter(
    highlight
).mark_line().encode(
    alt.X("date"),
    alt.Y("price"),
    alt.Color("symbol")
).add_params(
    highlight
)
  ```
</details>

* Use the `fields` or the `encodings` keyword argument (both will work, but the usage is different) so that your selection predicate will select all rows corresponding to the same company as the chosen point.
* Now if you click a point, we should get a zoomed in view of the entire line corresponding to that company.
* Adjust the color encoding so that, even after filtering away other companies, the colors stay the same.  Do this by specifying an explicit `domain`.


Here is a static snapshot of the specified chart after clicking on a point belonging to AMZN.

![](../resources/images/part3/stocks4.png)

<details>
  <summary>(Show Answer)</summary>

  ```python
highlight = alt.selection_point(fields=["symbol"])

alt.Chart(source).transform_filter(
    highlight
).mark_line().encode(
    alt.X("date"),
    alt.Y("price"),
    alt.Color("symbol").scale(domain=sorted(source["symbol"].unique()))
).add_params(
    highlight
)
  ```
</details>


The effect upon clicking is still a little jarring.  **Important takeaway**: Do not filter and select simultaneously, instead use one portion for the filtering and one portion for the selcting.



* Previously, we were filtering the data.  Remove the `transform_filter` portion and instead, use `alt.condition` together with our selection parameter to display the selected points in a larger `size`.  (Use the `size` encoding.)
* Use the `empty` keyword argument so that by default, all lines are displayed using the smaller size.
* **Warning**.  As mentioned above, you need to click precisely on a data point to have the change take effect, not on a line segment connecting two data points.  We will simplify this aspect below.


Here is a static snapshot of the specified chart after clicking on a point belonging to AAPL.

![](../resources/images/part3/stocks5.png)

<details>
  <summary>(Show Answer)</summary>

  ```python
highlight = alt.selection_point(fields=["symbol"], empty=False)

alt.Chart(source).mark_line().encode(
    alt.X("date"),
    alt.Y("price"),
    alt.Color("symbol"),
    size=alt.condition(
        highlight,
        alt.value(3),
        alt.value(1)
    )
).add_params(
    highlight
)
  ```
</details>


* Set the keyword argument `on="pointerover"` when defining our selection parameter.  Clicking is now no longer necessary (but you still need to be precisely over a data point).

<details>
  <summary>(Show Answer)</summary>

  ```python
highlight = alt.selection_point(
    fields=["symbol"],
    empty=False,
    on="pointerover"
)

alt.Chart(source).mark_line().encode(
    alt.X("date"),
    alt.Y("price"),
    alt.Color("symbol"),
    size=alt.condition(
        highlight,
        alt.value(3),
        alt.value(1)
    )
).add_params(
    highlight
)
  ```
</details>


* Things are still not very smooth because you need to be precisely over a data point for the change to take effect.  A nice effect can (eventually) be achieved by adding the keyword argument `nearest=True` to our selection parameter definition.  Try this.
* **Warning**.  This will break our existing functionality.

<details>
  <summary>(Show Answer)</summary>

  ```python
highlight = alt.selection_point(
    fields=["symbol"],
    empty=False,
    on="pointerover",
    nearest=True
)

alt.Chart(source).mark_line().encode(
    alt.X("date"),
    alt.Y("price"),
    alt.Color("symbol"),
    size=alt.condition(
        highlight,
        alt.value(3),
        alt.value(1)
    )
).add_params(
    highlight
)
  ```
</details>


* Try the same code as above, but now with a scatter plot instead of a line plot.  The functionality should be restored.

![](../resources/images/part3/stocks6.png)

<details>
  <summary>(Show Answer)</summary>

  ```python
highlight = alt.selection_point(
    fields=["symbol"],
    empty=False,
    on="pointerover",
    nearest=True
)

alt.Chart(source).mark_point().encode(
    alt.X("date"),
    alt.Y("price"),
    alt.Color("symbol"),
    size=alt.condition(
        highlight,
        alt.value(3),
        alt.value(1)
    )
).add_params(
    highlight
)
  ```
</details>



Our goal is now to achieve this scatter plot functionality with a line chart. The trick is that we will use an invisible scatter plot to make the selection, and then we will apply the condition to our line chart.

Adapt the above code in the following ways.
* Define two charts, one scatter plot and one line chart.  Name these charts `points` and `lines`, respectively.  Use a `base` definition so that you repeat as little code as possible.
* Make the scatter plot invisible by specifying an appropriate opacity value.
* Add our selection parameter to only one of the charts.  Which one?  (For which of the two charts does the `nearest=True` selection functionality work as expected?)
* Even though we added the selection parameter to only one of the two charts, we can still use it within a condition of the other chart (as long as the two charts are displayed together in some way, in our case, they will be layered on top of each other).
* Layer the scatter plot and the line chart on top of each other.


Here is a static snapshot of the desired chart after hovering on a point belonging to AAPL.

![](../resources/images/part3/stocks7.png)

<details>
  <summary>(Show Answer)</summary>

  ```python
highlight = alt.selection_point(
    fields=["symbol"],
    empty=False,
    on="pointerover",
    nearest=True
)

base = alt.Chart(source).encode(
    alt.X("date"),
    alt.Y("price"),
    alt.Color("symbol")
)

points = base.mark_point(opacity=0).add_params(
    highlight
)

lines = base.mark_line().encode(
    size=alt.condition(
        highlight,
        alt.value(3),
        alt.value(1)
    )
)

points+lines
  ```
</details>
